In [3]:
import time

x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

def calc_pure_python(desired_width,max_iterations):
    """복소 좌표(zs)와 복소 인자(cs)리스트를 만들고
    줄리아 집합을 생성한다."""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2: 
        x.append(xcoord)
        xcoord += x_step

    # 좌표리스트와 각 셀의 초기 조건 만들기
    # 초기 조건은 상수, 쉽게 제거 가능
    # 우리가 만든 함수의 몇몇 입력을 사용한 실제 시나리오를 시뮬레이션할 때 사용
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))
    print("Length of x:",len(x))
    print("Total elements:",len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs,cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + "took",secs,"seconds")

    # 다음 sum은 1000^2 그리드에 반복 300번을 가정한 값
    # 우리가 의도한 대로 좌표가 변화하는지 확인
    assert sum(output) == 33219980 # 결과값이 기댓과가 같은지?


    # CPU를 집중적으로 사용하는 계산함수
def calculate_z_serial_purepython(maxiter,zs,cs):
    """줄리아 갱신 규칙을 사용해서 output 리스트 계산하기"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output


In [2]:
import dis
import julia1
dis.dis(julia1.calculate_z_serial_purepython)

 48           0 LOAD_CONST               1 (0)
              2 BUILD_LIST               1
              4 LOAD_GLOBAL              0 (len)
              6 LOAD_FAST                1 (zs)
              8 CALL_FUNCTION            1
             10 BINARY_MULTIPLY
             12 STORE_FAST               3 (output)

 49          14 LOAD_GLOBAL              1 (range)
             16 LOAD_GLOBAL              0 (len)
             18 LOAD_FAST                1 (zs)
             20 CALL_FUNCTION            1
             22 CALL_FUNCTION            1
             24 GET_ITER
        >>   26 FOR_ITER                74 (to 102)
             28 STORE_FAST               4 (i)

 50          30 LOAD_CONST               1 (0)
             32 STORE_FAST               5 (n)

 51          34 LOAD_FAST                1 (zs)
             36 LOAD_FAST                4 (i)
             38 BINARY_SUBSCR
             40 STORE_FAST               6 (z)

 52          42 LOAD_FAST                2 (cs)
          

In [3]:
# 같은 일을 하지만 다른 결과
def fn_expressive(upper=1_000_000):
    total = 0
    for n in range(upper):
        total += n # total.__add__
    return total

def fn_terse(upper = 1_000_000):
    return sum(range(upper))

assert fn_expressive() == fn_terse(), "Expect identical results from both functions"

In [9]:
%timeit fn_expressive()
%timeit fn_terse()

64.8 ms ± 684 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
52.5 ms ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
import dis
dis.dis(fn_expressive)

  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (total)

  3           4 LOAD_GLOBAL              0 (range)
              6 LOAD_FAST                0 (upper)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                12 (to 26)
             14 STORE_FAST               2 (n)

  4          16 LOAD_FAST                1 (total)
             18 LOAD_FAST                2 (n)
             20 INPLACE_ADD
             22 STORE_FAST               1 (total)
             24 JUMP_ABSOLUTE           12

  5     >>   26 LOAD_FAST                1 (total)
             28 RETURN_VALUE


In [12]:
dis.dis(fn_terse)

  8           0 LOAD_GLOBAL              0 (sum)
              2 LOAD_GLOBAL              1 (range)
              4 LOAD_FAST                0 (upper)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 RETURN_VALUE
